# TP3 : Word2vec

## Imports

In [1]:
import pickle
import sys
import os

import pickle
import nltk
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

# config

temp_path = '../data/tmp/'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)

### Chargement et traitement des phrases du corpus

In [3]:
# Création d'un objet qui streame les lignes d'un fichier pour économiser de la RAM

class MySentences(object):
    """Tokenize and Lemmatize sentences""" #permet de tokenizer des "doubles" mots, par ex "premier" + "ministre" va être considéré comme un token "premier ministre"
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

infile = f"../data/sents.txt"
sentences = MySentences(infile)

#### Détection des bigrams

In [ ]:
# Création de l'objet 'phrases' = "dictionnaire d'expressions multi-mots associées à un score", dont les clés correspondent aux termes du corpus

bigram_phrases = Phrases(sentences)

In [ ]:
# Conversion des objets 'phrases' en objet 'phraser' = version light du 'phrases' -> convertit certains unigrams en bigrams s'ils sont pertinents

bigram_phraser = Phraser(phrases_model=bigram_phrases)

# Sauvegarder l'objet Phraser des bigrammes

with open(f"{temp_path}bigrams.p", 'wb') as f:
    pickle.dump(bigram_phraser, f)

#### Détection des trigrams

In [ ]:
# Ouvrir l'objet Phraser des bigrammes

bigram_phraser = pickle.load(open(f"{temp_path}bigrams.p", "rb"))

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
# Conversion en objet Phraser et sauvegarde du Phraser des trigrammes

trigram_phraser = Phraser(phrases_model=trigram_phrases)

with open(f"{temp_path}trigrams.p", 'wb') as f:
    pickle.dump(trigram_phraser, f)

#### Créations d'un corpus d'unigrams, bigrams, trigrams

In [4]:
bigram_phraser = pickle.load(open(f"{temp_path}bigrams.p", "rb"))
trigram_phraser = pickle.load(open(f"{temp_path}trigrams.p", "rb"))

In [5]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

# Imprimer une liste de n-grammes, qu'on répère car ils sont séparés par des _

print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

In [ ]:
# Sauvegarder cette liste dans le dossier temporaire
# disabled cause not working

# with open(f"{temp_path}ngram_corpus.p", 'wb') as f:
#    pickle.dump(corpus, f)

### Entraînement d'un modèle word2vec sur ce corpus

In [ ]:
# Charger la liste des n-grams
# disabled cause not working

# corpus = pickle.load(open("ngrams_list.p", "rb"))

In [64]:
%%time
model = Word2Vec( # default parameters
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte" (avant/après le mot)
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 'min_count' fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en plusieurs threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs
)

# Sauver le modèle dans un fichier
outfile = f"{temp_path}newspapers_window5_mincount5.model"
model.save(outfile)

CPU times: total: 6min 53s
Wall time: 40min 29s


In [62]:
%%time
model = Word2Vec( # smaller window, higher min_count
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=3, # La taille du "contexte" (avant/après le mot)
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 'min_count' fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en plusieurs threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs
)

# Sauver le modèle dans un fichier
outfile = f"{temp_path}newspapers_window3_mincount10.model"
model.save(outfile)

CPU times: total: 5min 47s
Wall time: 34min 51s


In [63]:
%%time
model = Word2Vec( # higher min_count and window
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte" (avant/après le mot)
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 'min_count' fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en plusieurs threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs
)

# Sauver le modèle dans un fichier
outfile = f"{temp_path}newspapers_window10_mincount10.model"
model.save(outfile)

CPU times: total: 6min 27s
Wall time: 31min 30s


In [120]:
%%time
model = Word2Vec( # higher min_count and window
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte" (avant/après le mot)
    min_count=20, # On ignore les mots qui n'apparaissent pas au moins 'min_count' fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en plusieurs threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs
)

# Sauver le modèle dans un fichier
outfile = f"{temp_path}newspapers_window20_mincount20.model"
model.save(outfile)

CPU times: total: 7min 9s
Wall time: 23min 23s


### Exploration des modèles

Comme proposé dans les consignes du TP, j'ai créé trois modèles jouant sur les paramètres 'min_count' et 'window'. Comme détaillé dans mon rapport, les différences semblent minimes.

In [5]:
# Charger les modèles en mémoire

models = []

models.append(Word2Vec.load(f"{temp_path}newspapers_window5_mincount5.model"))
models.append(Word2Vec.load(f"{temp_path}newspapers_window3_mincount10.model"))
models.append(Word2Vec.load(f"{temp_path}newspapers_window10_mincount10.model"))
models.append(Word2Vec.load(f"{temp_path}newspapers_window20_mincount20.model"))

nb_models = len(models)

In [47]:
# Fonction "similarity" : Calculer la similarité entre deux termes

first_term = ["journaliste", "chaud", "appartement", "socialiste", "chanter", "arbre", "enseigner"]
second_term = ["presse", "froid", "louer", "chretien", "citroen", "branche", "theologiens"]
nb_similarity_examples = len(first_term)

for i in range (nb_similarity_examples) :
    for j in range (nb_models):
        print(f"'{first_term[i]}' et '{second_term[i]}' (modèle {j}) : {models[j].wv.similarity(first_term[i], second_term[i])}")
    print("")

'journaliste' et 'presse' (modèle 0) : 0.2497072070837021
'journaliste' et 'presse' (modèle 1) : 0.23073068261146545
'journaliste' et 'presse' (modèle 2) : 0.2886613607406616
'journaliste' et 'presse' (modèle 3) : 0.343855082988739

'chaud' et 'froid' (modèle 0) : 0.7280045747756958
'chaud' et 'froid' (modèle 1) : 0.6839326620101929
'chaud' et 'froid' (modèle 2) : 0.7436212301254272
'chaud' et 'froid' (modèle 3) : 0.7467522025108337

'appartement' et 'louer' (modèle 0) : 0.6763496398925781
'appartement' et 'louer' (modèle 1) : 0.6020632982254028
'appartement' et 'louer' (modèle 2) : 0.7371500730514526
'appartement' et 'louer' (modèle 3) : 0.691918671131134

'socialiste' et 'chretien' (modèle 0) : 0.8494765758514404
'socialiste' et 'chretien' (modèle 1) : 0.8513742089271545
'socialiste' et 'chretien' (modèle 2) : 0.8594279289245605
'socialiste' et 'chretien' (modèle 3) : 0.8347303867340088

'chanter' et 'citroen' (modèle 0) : -0.16730761528015137
'chanter' et 'citroen' (modèle 1) : -0.2

In [41]:
# Fonction "most_similar" : Chercher les mots les plus proches d'un terme donné

most_similar_word = ["peugeot", "enseigner", "rome", "bruxelles", "jardin", "habiter"]
nb_most_similar_examples = len(most_similar_word)

for i in range (nb_most_similar_examples) :
    for j in range (nb_models):
        print(f"Mots les plus semblables à '{most_similar_word[i]}' (modèle {j}) :", models[j].wv.most_similar(most_similar_word[i], topn=10))
    print("")

Mots les plus semblables à 'peugeot' (modèle 0) : [('imperia', 0.9286518692970276), ('citroen', 0.9228317141532898), ('ford', 0.9175989031791687), ('opel', 0.9095059037208557), ('buick', 0.9083935618400574), ('minerva', 0.9041520357131958), ('skoda', 0.9018157124519348), ('chevrolet', 0.9011731147766113), ('mercedes', 0.9002424478530884), ('simca', 0.8955201506614685)]
Mots les plus semblables à 'peugeot' (modèle 1) : [('imperia', 0.9339192509651184), ('chrysler', 0.9255465269088745), ('citroen', 0.9190285801887512), ('simca', 0.9189803004264832), ('volkswagen', 0.918897807598114), ('ford', 0.9178043007850647), ('taunus', 0.9104763865470886), ('buick', 0.910352349281311), ('cadillac', 0.9095398783683777), ('opel', 0.9092620611190796)]
Mots les plus semblables à 'peugeot' (modèle 2) : [('citroen', 0.9472780227661133), ('ford', 0.9429683089256287), ('chevrolet', 0.9148492813110352), ('imperia', 0.9125781059265137), ('minerva', 0.9059906005859375), ('opel', 0.9050105810165405), ('trlumph'